### CV-Optimizer - Experimentos de Lectura usando LLMs
- Comenzamos con OpenAI, DeepSeek y Gemini

- Este notebook sirve como entorno de experimentación para:
    - Leer archivo de curriculum (inicialmente en formato pdf)
    - Enviarlo a distintos LLMs para lectura desde prompt prediseñado [esto es lo que queremos testear en este notebook]

In [1]:
from dotenv import load_dotenv
import os


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


print("OpenAI:", bool(OPENAI_API_KEY))
print("DeepSeek:", bool(DEEPSEEK_API_KEY))
print("Gemini:", bool(GEMINI_API_KEY))

OpenAI: True
DeepSeek: True
Gemini: True


In [ ]:
# Si ya tienes genai instalado, desinstálalo primero
!pip install google-generativeai

In [13]:
import google.generativeai as genai


#openai_client = OpenAI(api_key=OPENAI_API_KEY)


genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")


In [14]:
from pypdf import PdfReader

def read_pdf(path):
    reader = PdfReader(path)
    return "\n".join(p.extract_text() for p in reader.pages)


def read_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

In [15]:
BASE_PROMPT = """
You are an expert technical recruiter.
Analyze the following CV and provide:
1. Strengths
2. Weaknesses
3. Suggestions for improvement


CV:
{cv}
"""

In [16]:
def analyze_openai(cv_text):
    response = openai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "user", "content": BASE_PROMPT.format(cv=cv_text)}],
    temperature=0.2
    )
    return response.choices[0].message.content


# ---- DeepSeek ----


def analyze_deepseek(cv_text):
    try:
        response = deepseek_client.chat.completions.create(
            model="deepseek-chat",
            messages=[{"role": "user", "content": BASE_PROMPT.format(cv=cv_text)}],
            temperature=0.2
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ DeepSeek error: {e}"



# ---- Gemini ----


def analyze_gemini(cv_text):
    response = gemini_model.generate_content(BASE_PROMPT.format(cv=cv_text))
    return response.text

In [17]:
cv_path = '../../data/raw/CV_gustavo_arcaya_ENG_test.pdf'

In [19]:
if cv_path.endswith(".pdf"):
    cv_text = read_pdf(cv_path)
else:
    cv_text = read_txt(cv_path)


MODEL = "gemini" # openai | deepseek | gemini


if MODEL == "openai":
    result = analyze_openai(cv_text)
elif MODEL == "deepseek":
    result = analyze_deepseek(cv_text)
elif MODEL == "gemini":
    result = analyze_gemini(cv_text)


print(result)

NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.